In [3]:
import pandas as pd
import numpy as np
import os
import pyspark
import pyspark.sql.functions as F

# average ratings
###  average rating per actors

In [ ]:
link_name_basics_titles = pd.read_parquet("../data/edited_data/datamart.db/link_name_basics_titles")
title_ratings = pd.read_parquet("../data/edited_data/datamart.db/title_ratings")
title_basics = pd.read_parquet("../data/edited_data/datamart.db/title_basics")

In [ ]:
link_name_basics_titles_start_year = (
    link_name_basics_titles
    .merge(title_basics[["tconst", "startYear"]], left_on="tconst", right_on="tconst")
)

In [ ]:
name_avg_ratings = (
    link_name_basics_titles_start_year
    .merge(
        link_name_basics_titles_start_year.rename(columns={"tconst": "_tconst", "startYear": "_startYear"}), 
        right_on="nconst", left_on="nconst")
    .merge(title_ratings.rename(columns={"tconst": "_tconst"}), right_on="_tconst", left_on="_tconst")
)

name_avg_ratings = name_avg_ratings[name_avg_ratings._startYear < name_avg_ratings.startYear]
name_avg_ratings = name_avg_ratings.groupby(["tconst", "nconst"])[["averageRating", "numVotes"]].mean()
name_avg_ratings = name_avg_ratings.rename(
    columns={
        "averageRating": "averageRating_name",
        "numVotes": "numVotes_name",
    }
)
name_avg_ratings.head()

In [ ]:
title_name_avg_ratings = name_avg_ratings.groupby("tconst")[["averageRating_name", "numVotes_name"]].mean()
title_name_avg_ratings.head()

In [ ]:
title_name_avg_ratings.to_parquet("../data/edited_data/intermediate.db/title_name_avg_ratings")

### average rating per genre

In [ ]:
title_basics = pd.read_parquet("../data/edited_data/datamart.db/title_basics")
title_ratings = pd.read_parquet("../data/edited_data/datamart.db/title_ratings")

title_basics.set_index("tconst", inplace=True)
title_ratings.set_index("tconst", inplace=True)

In [ ]:
genre_ratings = title_basics[["titleType"]].join(title_ratings, how="left")

genres_avg_ratings = genre_ratings.groupby(["titleType"]).mean()
genres_avg_ratings = genres_avg_ratings.rename(
    columns={
        "averageRating": "averageRating_genres", 
        "numVotes": "numVotes_genres"
    }
)

genres_avg_ratings.head()

In [ ]:
title_genres_avg_ratings = title_basics[["titleType"]].join(genres_avg_ratings, on="titleType")
title_genres_avg_ratings.head()

In [ ]:
title_genres_avg_ratings.to_parquet("../data/edited_data/intermediate.db/title_genres_avg_ratings")

### average rating per crew

In [86]:
title_principals = pd.read_parquet("../data/edited_data/datamart.db/title_principals")
link_name_basics_titles = pd.read_parquet("../data/edited_data/datamart.db/link_name_basics_titles")
title_ratings = pd.read_parquet("../data/edited_data/datamart.db/title_ratings")
title_basics = pd.read_parquet("../data/edited_data/datamart.db/title_basics")

In [87]:
link_name_basics_titles_start_year = (
    link_name_basics_titles
    .merge(title_basics[["tconst", "startYear"]], left_on="tconst", right_on="tconst")
)

In [88]:
crew_avg_ratings_raw = (
    link_name_basics_titles_start_year
    .merge(
        link_name_basics_titles_start_year.rename(columns={"tconst": "_tconst", "startYear": "_startYear"}), 
        right_on="nconst", left_on="nconst")
    .merge(title_ratings.rename(columns={"tconst": "_tconst"}), right_on="_tconst", left_on="_tconst")  
)    

In [89]:
crew_avg_ratings_raw.head()

,nconst,tconst,startYear,_tconst,_startYear,averageRating,numVotes
0,nm0000198,tt4555426,2017.0,tt4555426,2017.0,7.4,140294
1,nm0000198,tt1340800,2011.0,tt4555426,2017.0,7.4,140294
2,nm0000198,tt0468569,2008.0,tt4555426,2017.0,7.4,140294
3,nm0000198,tt0103874,1992.0,tt4555426,2017.0,7.4,140294
4,nm7322064,tt4555426,2017.0,tt4555426,2017.0,7.4,140294


In [90]:
crew_avg_ratings_raw = crew_avg_ratings_raw.merge(
    title_principals[["category","nconst"]], 
    right_on=["tconst","nconst"], 
    left_on=["tconst","nconst"]) 

In [91]:
crew_avg_ratings_raw.head()

,nconst,tconst,startYear,_tconst,_startYear,averageRating,numVotes,category
0,nm0000198,tt4555426,2017.0,tt4555426,2017.0,7.4,140294,actor
1,nm0000198,tt4555426,2017.0,tt1340800,2011.0,7.1,176138,actor
2,nm0000198,tt4555426,2017.0,tt0468569,2008.0,9.0,2093112,actor
3,nm0000198,tt4555426,2017.0,tt0103874,1992.0,7.4,175613,actor
4,nm0000198,tt1340800,2011.0,tt4555426,2017.0,7.4,140294,actor


In [92]:
crew_avg_ratings_raw = crew_avg_ratings_raw[crew_avg_ratings_raw._startYear < crew_avg_ratings_raw.startYear]

In [93]:
crew_avg_ratings_raw = crew_avg_ratings_raw.groupby(["tconst", "nconst","category"])[["averageRating", "numVotes"]].mean()
crew_avg_ratings = crew_avg_ratings_raw.groupby(["tconst","category"])[["averageRating", "numVotes"]].mean()

In [94]:
categories = {"actor": "actor",
"actress": "actor",
"archive_footage": "other",
"archive_sound": "other",
"cinematographer": "other",
"composer": "other",
"director": "director",
"editor": "other",
"producer": "producer",
"production_designer": "other",
"self": "actor",
"writer": "other"
}

In [95]:
#Run only once
crew_avg_ratings = crew_avg_ratings.reset_index(level=1)

In [96]:
crew_avg_ratings['category_agg'] = crew_avg_ratings["category"].apply(lambda x: categories[x])
crew_avg_ratings.head()

,category,averageRating,numVotes,category_agg
tconst,,,,
tt0002143,writer,5.90,29.0,other
tt0011396,writer,5.60,24.0,other
tt0012025,writer,5.85,24.0,other
tt0015268,director,7.00,757.0,director
tt0015650,director,7.30,1394.0,director


In [97]:
crew_avg_ratings_final = crew_avg_ratings.groupby(["tconst",'category_agg'])[["averageRating", "numVotes"]].mean()
crew_avg_ratings_final.head()

,,averageRating,numVotes
tconst,category_agg,,
tt0002143,other,5.90,29.0
tt0011396,other,5.60,24.0
tt0012025,other,5.85,24.0
tt0015268,director,7.00,757.0
tt0015650,director,7.30,1394.0


In [99]:
crew_avg_ratings_final = (
    crew_avg_ratings_final
    .reset_index()
    .pivot(index='tconst',columns='category_agg',values='averageRating')
    .rename(columns={"averageRating": "averageRating_crewCategory"})
)

In [100]:
crew_avg_ratings_final.head(10)

category_agg,actor,director,other,producer
tconst,,,,
tt0002143,NaN,NaN,5.90,NaN
tt0011396,NaN,NaN,5.60,NaN
tt0012025,NaN,NaN,5.85,NaN
tt0015268,NaN,7.00,NaN,NaN
tt0015650,NaN,7.30,NaN,NaN
tt0015848,NaN,7.30,NaN,NaN
tt0016669,NaN,NaN,6.90,NaN
tt0017136,NaN,NaN,7.90,NaN
tt0017850,NaN,7.05,NaN,NaN


In [102]:
crew_avg_ratings_final.to_parquet("../data/edited_data/intermediate.db/title_avg_ratings_crew_category")

# budget
### Link movie.budget to title.basics

In [ ]:
import pyspark
import os
import pandas as pd

In [ ]:
movie_budget = pd.read_parquet("../data/parsed_data/movie_budget")
title_basics = pd.read_parquet("../data/edited_data/datamart.db/title_basics")

In [ ]:
movie_budget.info()

In [ ]:
movie_budget["title"] = movie_budget.Movie.str.lower().replace("\W", "", regex=True)
movie_budget = movie_budget.set_index("title")

In [ ]:
movie_budget.head()

In [ ]:
title_basics["title"] = title_basics.originalTitle.str.lower().replace("\W", "", regex=True)
title_basics = title_basics[title_basics.titleType == "movie"]
title_basics = title_basics.set_index("title")

In [ ]:
title_basics.head()

In [ ]:
title_basics["titleBasicsFlag"] = True

title_budget = (
    movie_budget
    .join(title_basics[["titleBasicsFlag", "tconst"]], how="left")
)

In [ ]:
title_basics[title_basics.tconst.isin(["tt5671400", "tt2388621", "tt7725384"])]

In [ ]:
title_budget.head(10)

In [ ]:
title_budget.titleBasicsFlag.value_counts()

In [ ]:
n_obs = pd.DataFrame(title_budget.groupby(title_budget.index).size()).rename(columns={0: "nObs"})
title_budget = title_budget.join(n_obs)
title_budget = title_budget[title_budget.nObs == 1]

In [ ]:
title_budget.titleBasicsFlag.value_counts()

In [ ]:
title_budget = title_budget.reset_index().set_index("tconst")

In [ ]:
title_budget.to_parquet("../data/edited_data/intermediate.db/title_budget")

### primary_title

In [ ]:
title_basics = pd.read_parquet("../data/edited_data/datamart.db/title_basics")
title_basics.set_index("tconst", inplace=True)

In [ ]:
primary_title = title_basics[["primaryTitle"]]
primary_title["nLetters"] = primary_title.primaryTitle.str.len()
primary_title["nCapitalLetters"] = primary_title.primaryTitle.apply(lambda x: sum(1 for c in x if c.isupper()))
primary_title["nWords"] = primary_title.primaryTitle.str.split("\W").str.len()

In [ ]:
primary_title.head()

In [ ]:
primary_title.to_parquet("../data/edited_data/intermediate.db/primary_title")